# intro


The purpose of the project is to learn the mapping from polar coordinates to a a discrete 10x10 grid of cells in the plane, using a neural network. 

The supervised dataset is given to you in the form of a generator (to be considered as a black box).

The model must achieve an accuracy of 95%, and it will be evaluated in a way **inversely proportional to the number of its parameters: the smaller, the better.**

**WARNING**: Any solution taking advantage of meta-knowledge about the generator will be automatically rejected.

In [55]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Reshape, concatenate
from tensorflow.keras.models import Model
from tensorflow import keras
#check if gpu is present
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

Here is the generator. It returns triples of the form ((theta,rho),out) where (theta,rho) are the polar coordinates of a point in the first quadrant of the plane, and out is a 10x10 map with "1" in the cell corresponding to the point position, and "0" everywhere else.

By setting flat=True, the resulting map is flattened into a vector with a single dimension 100. You can use this variant, if you wish. 

In [56]:
def polar_generator(batchsize,grid=(10,10),noise=.002,flat=False):
  while True:
    x = np.random.rand(batchsize)
    y = np.random.rand(batchsize)
    out = np.zeros((batchsize,grid[0],grid[1]))
    xc = (x*grid[0]).astype(int)
    yc = (y*grid[1]).astype(int)
    for b in range(batchsize):
      out[b,xc[b],yc[b]] = 1
    #compute rho and theta and add some noise
    rho = np.sqrt(x**2+y**2) + np.random.normal(scale=noise)
    theta = np.arctan(y/np.maximum(x,.00001)) + np.random.normal(scale=noise)
    if flat:
      out = np.reshape(out,(batchsize,grid[0]*grid[1]))
    yield ((theta,rho),out)

Let's create an instance of the generator on a grid with dimension 3x4

In [57]:
g1,g2 = 3,4
gen = polar_generator(1,grid=(g1,g2),noise=0.0)

And now let's see a few samples.

In [58]:
(theta,rho),maps = next(gen)
for i,map in enumerate(maps):
  #let us compute the cartesian coordinates
  x = np.cos(theta[i])*rho[i]
  y = np.sin(theta[i])*rho[i]
  print("x coordinate (row): {}".format(int(x*g1)))
  print("y coordinate (col): {}".format(int(y*g2)))
  print("map:")
  print(np.reshape(map,(g1,g2)))

x coordinate (row): 0
y coordinate (col): 2
map:
[[0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Exercise: add noise to the generator, and check the effect on the "ground truth".

## What to deliver

For the purposes of the project you are supposed to work with the **default 10x10 grid, and the default noise=.002**

The generator must be treatead as a black box, do not tweak it, and do not exploit its semantics that is supposed to be unknown. You are allowed to work with the "flat" modality, if you prefer so.

You need to:
1.   define an accuracy function (take inspiration from the code of the previous cell)
2.   define a neural network taking in input theta and rho, and returning out
3. measure the network's accuracy that must be above 95% (accuracy must be evaluated over at least 20000 samples)
4. tune the network trying to decrease as much as possible the numer of parameters, preserving an accuracy above 95%. Only your best network must be delivered.

You must deliver a SINGLE notebook working on colab, containing the code of the network, its summary, the training history, the code for the accurary metric and its evaluation on the network.

**N.B.** The accuracy must be above 95% but apart from that it does not influence the evaluation. You score will only depend on the number of parameters: the lower, the better.






# Model

In [59]:
from keras.models import Model
from keras.layers import Input, Concatenate, Dense,  BatchNormalization, Conv2D, UpSampling2D
from keras.activations import leaky_relu
import keras.backend as K
batch_size=6500

def create_model():
    # Define the input layers
    theta_input = Input(shape=(1,), name='theta_input')
    rho_input = Input(shape=(1,), name='rho_input')
    # Concatenate the inputs
    concatenated = Concatenate()([theta_input, rho_input])

    # HIDDEN LAYERS
    hidden=Dense(6,  activation="selu")(concatenated)
    hidden=Dense(6,  activation="selu")(hidden)
    hidden=Dense(2,  activation="selu")(hidden)

    output=Dense(100,  activation="softmax")(hidden)



    # Define the model and return it
    model = Model(inputs=[theta_input, rho_input], outputs=output)
    return model

In [60]:
def one_hot_accuracy(y_true, y_pred):
  # Convert the predictions to one_hot vectors
  y_pred_one_hot = K.one_hot(K.argmax(y_pred, axis=1), K.shape(y_pred)[1])
  # Compare the predictions with the true labels and count the matches
  correct = K.cast(K.equal(y_true, y_pred_one_hot), K.floatx())
  matches = K.sum(correct, axis=-1)
  # Divide the matches by the number of classes to get the accuracy
  accuracy = matches / K.cast(K.shape(y_pred)[1], K.floatx())
  return accuracy

In [61]:
model=create_model()
# from tensorflow import keras
opt = keras.optimizers.Adamax(learning_rate=0.1)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["categorical_accuracy"])
model.summary()



Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 theta_input (InputLayer)       [(None, 1)]          0           []                               
                                                                                                  
 rho_input (InputLayer)         [(None, 1)]          0           []                               
                                                                                                  
 concatenate_6 (Concatenate)    (None, 2)            0           ['theta_input[0][0]',            
                                                                  'rho_input[0][0]']              
                                                                                                  
 dense_24 (Dense)               (None, 6)            18          ['concatenate_6[0][0]']    

In [62]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Define the EarlyStopping callback
early_stop = EarlyStopping(monitor='val_categorical_accuracy', patience=25, verbose=1, mode='max', restore_best_weights=True)

# Define the ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.1, patience=5, verbose=1, mode='max', min_lr=1e-5)

# Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint('polar_express.h5', monitor='val_categorical_accuracy', save_best_only=True, mode='max', verbose=0)

In [63]:
batch_size=2048
# Define the generator instance
train_gen = polar_generator(batchsize=batch_size, flat=True)
val_gen=polar_generator(batchsize=batch_size, flat=True)
# Train the model using the generator
history=model.fit(train_gen, steps_per_epoch=2000, epochs=200, callbacks=[early_stop, reduce_lr, checkpoint], validation_data=val_gen, validation_steps=250, shuffle=True)

Epoch 1/200
2000/2000 [==============================] - 15s 6ms/step - loss: 1.9046 - categorical_accuracy: 0.4070 - val_loss: 1.2172 - val_categorical_accuracy: 0.6257 - lr: 0.1000
Epoch 2/200
2000/2000 [==============================] - 13s 7ms/step - loss: 1.0542 - categorical_accuracy: 0.6574 - val_loss: 0.9001 - val_categorical_accuracy: 0.7107 - lr: 0.1000
Epoch 3/200
2000/2000 [==============================] - 13s 6ms/step - loss: 0.8146 - categorical_accuracy: 0.7292 - val_loss: 0.7990 - val_categorical_accuracy: 0.7041 - lr: 0.1000
Epoch 4/200
2000/2000 [==============================] - 13s 6ms/step - loss: 0.6913 - categorical_accuracy: 0.7680 - val_loss: 0.7115 - val_categorical_accuracy: 0.7369 - lr: 0.1000
Epoch 5/200
2000/2000 [==============================] - 13s 6ms/step - loss: 0.6027 - categorical_accuracy: 0.7911 - val_loss: 0.6326 - val_categorical_accuracy: 0.7836 - lr: 0.1000
Epoch 6/200
2000/2000 [==============================] - 13s 7ms/step - loss: 0.5272 

KeyboardInterrupt: ignored

In [ ]:
gen=polar_generator(batchsize=20000, flat=True)
data=next(gen)
pred=model.predict(data[0], verbose=False)
acc=one_hot_accuracy(data[1], pred)
print("accuracy:", np.mean(acc))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
# Plot the training loss
plt.subplot(1,2,1)
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

# Plot the training accuracy
plt.subplot(1,2,2)
plt.plot(history.history['val_categorical_accuracy'])
plt.plot(history.history['categorical_accuracy'])
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()